# Technical note: OpenCL simulation

These technical notes describe the OpenCL simulation implemented in Myokit.
This is designed for rectangular grids (or more complicated meshes) of _coupled cells,_ connected via a _diffusion current_.
It is also possible to disable the diffusion current, and use it to perform single-cell simulations in parallel.

We assume each cell model contains a membrane potential variable, defined by an equation:

\begin{equation}
\frac{d V}{d t}=-\frac{1}{C_{m}}(i_{\mathit{ion}}+i_{\mathit{stim}}+i_{\mathit{diff}})\label{eq:dv/dt}
\end{equation}

where $i_\mathit{diff}$ is a diffusion current

\begin{equation}
i_{\mathit{diff},i}=\sum_{j\in N_{i}}g_{ij}\left(V_{i}-V_{j}\right)\label{eq:idiff}
\end{equation}

and 

- $N_{i}$ is the set of neighbours of cell $i$;
- $g_{ij}$ is the conductance between cell $i$ and its neighbour $j$.

## Specifying conductances

As of version 1.33.0, Myokit provides three ways of specifying the conductances $g_{ij}$.

### Rectangular, homogeneous conductances

In this mode, we assume either a "strand" (in 1d) or a rectangular grid (2d) of cells, connected with a single conductance $g_x$ (1d) or a horizontal and a vertial conductance $(g_x, g_y)$ (2d).

### Rectangular, heterogeneous conductances

In this mode, we still assume a strand or a rectangular grid, but each conductance is set separately.

In the 1d case, for $n$ cells we, this means we need a list of $n - 1$ conductances.

In the 2d case, for $(m \times n)$ cells, we need two matrices. One of $(m - 1 \times n)$ conductances in the direction of the first coordinate, and one of $(m \times n - 1)$ conductances in the direction of the second coordinate.

### Arbitrary geometries

In this mode, the simulation is initialised in 1d mode, so that each cell is identified by a single subscript $i$ or $j$.
Next, the use specifies a list of connections

## Heterogeneous parameters

The `SimulationOpenCL` allows _literal constants_ (constants that do not depend on other constants) to be set using a _scalar field_, i.e. to specify a value per cell.

## Heterogeneous pacing

In most simulations, only a few cells are paced.
The `SimulationOpenCL` allows the paced cells to be specificied either explicitly (using a list of cell indices) or by specifying a rectangle of paced cells.

All paced cells receive the same pacing signal.
However, it is possible to set up more complex pacing e.g. by defining a stimulus current as $I_i = p_i * u(t)$ and setting $p_i$ to vary from cell to cell.

## Relationship to single-cell notation

As before, we can write the system of ODEs for each cell as:

\begin{equation}
\dot{y}(t)=f\left(y(t),u(t),t, p\right)
\end{equation}

but we now need to take into account that a cell i's external inputs $u_i$ depends on $y_i$, plus those of its neighbours:

\begin{equation}
\dot{y_i}(t)=f\left(y_i(t),u_i(y, t), t, p_i\right)
\end{equation}

Here we write $f$ for each cell, instead of $f_i$, to indicate the function is the same (we use the same model for every cell).

However, we write $u_i$ to indicate that pacing can be applied to selected cells only, and $p_i$ to indicate the possible use of scalar fields.

## Relationship to mono-domain equations

The bidomain model starts from the assumption of ohmic conductance currents as in the equation above.
It then adds a few components representing the extracellular conductance and generalises the result to a continuous PDE.
The mono-domain is derived from the bi-domain model by dropping the terms explicitly modelling the extracellular conductance.
Many modelers then assume only conduction in the principal directions (x,y and z) and add the boundary condition that no current enters or leaves the system at the sides.
Then, using a finite-difference approximation of the second order derivatives arising from the bidomain equations,
we arrive right back at the original equation specifying ohmic conductance between neighbouring cells.

While the PDE form is highly advantageous when discretising to an arbitrary-shaped mesh, for simple cell-to-cell conductance models it has the disadvantage of introducing extra parameters (with perhaps a less clear meaning than $g_{x}$ and $g_{y}$).
However, in order to compare results in Myokit with mono-domain simulations, it may be necessary to translate beteen the used parameter sets.
Their relationship is derived below.

According to the mono-domain model, each point in the heart can be described as:

\begin{equation}
\frac{\lambda}{1+\lambda}\nabla\cdot\left(\Sigma_{i}\nabla v\right)=\chi\left(C_{m}\frac{\partial v}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\right)\label{eq:monodom}
\end{equation}

With:

- $\lambda$ The intra- to extracellular conductivity ratio
- $\Sigma_{i}$ A tensor representing the intracellular conductivity
- $v$ The local membrane potential
- $\chi$ The surface area of the membrane per unit volume
- $C_{m}$ Capacitance per unit area
- $i_{ion}$ Ionic current per unit area

Assuming all currents are normalised to unit area, we can combine this with the equation for $\frac{\partial V}{\partial t}$ to find the mono-domain equation for $i_{diff}$:

\begin{eqnarray*}
\frac{\lambda}{1+\lambda}\nabla\cdot\left(\Sigma_{i}\nabla V\right) & = & \chi\left(C_{m}\frac{\partial V}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\right)\\
\frac{\lambda}{\chi\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right) & = & C_{m}\frac{\partial V}{\partial t}+i_{\mathit{ion}}+i_{\mathit{stim}}\\
\frac{\partial V}{\partial t} & = & \frac{-1}{C_{m}}\left(i_{\mathit{ion}}+i_{\mathit{stim}}-\frac{1}{\chi}\frac{\lambda}{\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right)\right)\\
 & = & \frac{-1}{C_{m}}\left(i_{\mathit{ion}}+i_{\mathit{stim}}+i_{\mathit{diff}}\right)
\end{eqnarray*}

so that

\begin{equation}
i_{\mathit{diff}}=-\frac{1}{\chi}\frac{\lambda}{\left(1+\lambda\right)}\nabla\cdot\left(\Sigma_{i}\nabla V\right)
\end{equation}

For the 2D case, let

\begin{equation}
\Sigma_{i}=\sigma_{x}\hat{\mathbf{i}}+\sigma_{y}\hat{\mathbf{j}}
\end{equation}

then

\begin{equation}
\Sigma_{i}\nabla V=\frac{\partial V}{\partial x}\sigma_{x}\hat{\mathbf{i}}+\frac{\partial V}{\partial y}\sigma_{y}\hat{\mathbf{j}}
\end{equation}

and

\begin{equation}
\nabla\cdot\left(\Sigma_{i}\nabla V\right)=\frac{\partial^{2}V}{\partial x^{2}}\sigma_{x}+\frac{\partial^{2}V}{\partial y^{2}}\sigma_{y}
\end{equation}

A finite difference approximation of $\frac{\partial^{2}V}{\partial x^{2}}$ is

\begin{eqnarray*}
\frac{\partial^{2}V}{\partial x^{2}} & = & \frac{V_{i-1}-2V_{i}+V_{i+1}}{\Delta x^{2}}\\
 & = & -\left(\frac{V_{i}-V_{i-1}}{\Delta x^{2}}+\frac{V_{i}-V_{i+1}}{\Delta x^{2}}\right)\\
 & = & \frac{-1}{\Delta x^{2}}\sum_{j\in N_{x}}\left(V_{i}-V_{x}\right)
\end{eqnarray*}

where $N_{x}$ is the set of neighbours of cell $i$ in the $x$ direction.
Using this approximation, we find:

\begin{eqnarray*}
i_{\mathit{diff}} & = & \frac{\sigma_{x}}{\chi\Delta x^{2}}\frac{\lambda}{1+\lambda}\sum_{j\in N_{x}}\left(V_{i}-V_{x}\right)+\frac{\sigma_{y}}{\chi\Delta y^{2}}\frac{\lambda}{1+\lambda}\sum_{j\in N_{y}}\left(V_{i}-V_{y}\right)\\
 & = & g_{x}\sum_{x\in N_{x}}\left(V_{i}-V_{x}\right)+g_{y}\sum_{y\in N_{y}}\left(V_{i}-V_{y}\right)\\
 & = & \sum g_{ij}\left(V_{i}-V_{j}\right)
\end{eqnarray*}

So to convert the four mono-domain parameters to a single conductance, we use:
\begin{eqnarray*}
g_{x} & = & \frac{\lambda\sigma_{x}}{\chi\left(1+\lambda\right)\Delta x^{2}}\\
g_{y} & = & \frac{\lambda\sigma_{y}}{\chi\left(1+\lambda\right)\Delta y^{2}}
\end{eqnarray*}

## Solver

The solver used in the `SimulationOpenCL` class is a simple forward-Euler method (although Rush-Larsen updates can be used for Hodgkin-Huxley gating equations, if desired).
This has the advantage of being quite a _stable_ method, meaning it can take relatively large steps (larger than methods that take _slightly_ more complex explicit update steps, but not as large as _implicit_ methods).
It also has a low computational cost _per step_, requiring only a single evaluation of the RHS.
A major downside is the requirement to choose a good step size.

There are some plans to build an adaptive method in Myokit, still based on forward Euler.
However, it is worth noting that, as [Garcia-Molla et al.](https://doi.org/10.1016/j.compbiomed.2013.10.023) point out, multi-cell simulations are often of re-entrant arrhythmias in which there are always depolarising cells which require the smallest step size.
In their work, they found fixed step-size methods often outperformed adaptive ones.